In [256]:
import codecs
import re

In [257]:
# from https://segmentfault.com/q/1010000000732038
# to detect english character ONLY
def isAlpha(word):
    try:
        return word.encode('ascii').isalpha()
    except UnicodeEncodeError:
        return False


In [258]:
filename_cmu = "cmudict-0.7b"
f = open(filename_cmu,"r")
d = dict()
max_num = 0
for line in f.readlines():
    if line[0:3] != ";;;":
        line = line.split("  ",1)
        d[line[0]] = line[1]
        if line[0][-1] == ")":
            try: 
                num_cur = int(line[0][-2])      
                if int(line[0][-2]) > max_num:
                    max_num = num_cur
            except:
                continue
print("Finish loading cmu dictionary")

Finish loading cmu dictionary


In [259]:
dir = "./transcript"
filenames = os.listdir(dir)
text = []
for filename in filenames:
    if filename[-4:] == ".txt":
        with codecs.open(dir+"/"+filename, 'r', 'utf-8') as f:
            for line in f.readlines():
                text += [line]
            f.close()
            #print("Done "+ filename)
print("Finish reading lines")

Finish reading lines


In [260]:
words = []
line_waste_count = 0
d_unparse = dict()
idx = -1

for sentence in text:
    idx += 1
    #if "马来糕" in sentence:
    #    print(sentence)
    #    print(idx)
    sentence_origin = sentence
    sentence = sentence.split() # 1st element -- audio file id, 2nd element -- start time, 3rd element -- end time
    words_cur = sentence[4:]
    unparse = False
    
    idx_cur = -1
    for word in words_cur: # filter out concatenated characters having Chinese characters, with length over 6
        idx_cur += 1
        
        # deal with wrong transcription like co[mm]on (idx: 25799) and co[mm]unication (idx: 26347)
        if "]" in word and word[-1] != "]":
            word = word.replace("]","")
            word = word.replace("[","")

        # deal with pronuncisation of single letter like "i. t."(idx: 48495)
        if word[-1] == "." and len(word) == 2:
            word = word[0]
        
        words_cur[idx_cur] = word
    
        
        # filter out unparsed element (unparse = F -> unparse = T)
        # for example, [leh]每次上学的时候daddy都会买糕点给我吃买马来糕#kuih# (idx: 53628)
        try:
            d[word.upper()]
        except:
            word_split = word.split("-")
            if len(word_split) > 1: # pass if it is in the form of "a-b"
                continue
            elif len(word) < 15 and (word[-1] == "]" or word[-1] ==")" or word[-1] =="#" or word[0] == "[" or word[0] == "(" or word[0] == "#"): # pass if it's a non-transcribable sound
                continue
            elif word[-1] == "%": # %a% seems like special nouns, such as name (%chelsia%, idx 26701), game (%viwawa%, idx 21513)
                continue
            elif isAlpha(word.replace("'","")):
                continue
            elif isAlpha(word.replace('"',"")):
                continue
            elif isAlpha(word.replace(".","")):
                continue
            else:
                if len(word) > 6 and word.isalpha() == False:
                    unparse = True
        finally:
            if word[-1] == "]" or word[-1] ==")" or word[-1] =="#":
                line_waste_count += 1
                break
    
    if unparse:
        d_unparse[idx] = [sentence_origin]
    
    words += words_cur

print("Finish concatenating words")
print("Number of utterance: " + str(len(text)))
print("Number of utterance with unparsed elements: " + str(len(d_unparse)))
print("Number of utterance with non-transcribable sounds: " + str(line_waste_count))


Finish concatenating words
Number of utterance: 54929
Number of utterance with unparsed elements: 204
Number of utterance with non-transcribable sounds: 8823


In [261]:
d_unparse

{11754: ['33NC43FBQ_0101\t2551645\t2555415\tyou know  "sara\'s" speaking thing it got cancelled 两次 [eh]\n'],
 19959: ['NI22FBP_0101\t668052\t669342\t他  对  她  有  ·䞠\x84\x9f枠§\x89\n'],
 36795: ['UI03FAZ_0101\t520078\t526209\tso call 那些support 的人他们就会把那些啊p. o. 的informations\n'],
 44981: ['UI15FAZ_0104\t646953\t653869\t又可以穿的比较 sexy 一点咯有有男生去了#sei#有 #paiseh# [wo]\n'],
 45126: ['UI16MAZ_0101\t334474\t339800\t就 正式跟user有contact的就是p. c. service咯 那个修理电脑部门\n'],
 45129: ['UI16MAZ_0101\t367367\t370912\t啊 outlook或者其他s. a. p. \n'],
 45199: ['UI16MAZ_0101\t814554\t825136\t对 就像平常你们m. s. n. 或者 communicator 你跟朋友讲话 你可以同时间跟两个到三个讲话的嘛 \n'],
 45282: ['UI16MAZ_0102\t276713\t281922\t他们 对就是他们就是他们p. s. n. 咯就是 p. s.  play station network \n'],
 45419: ['UI16MAZ_0103\t145562\t151446\t但是有时他开始讲的那些故事拖拖一整集都没有内容的 就很#sien#咯\n'],
 45496: ['UI16MAZ_0103\t747433\t752878\t哦那个我还不能啦 但是我？了起来大约大约想想这样才可以啦 \n'],
 45518: ['UI16MAZ_0103\t902106\t910555\t基本上lens的话 基本上lens的话你越好的话[leh] 拍出来时越清的 不会 不会变成旧的\n'],
 45536: ['UI16MAZ_0103\t10461

In [262]:
text[32788] # cannot be understood by human

'NI61FBP_0101\t696201\t702891\t有  以前  我 sec one sec two 就 用 里 谷 join 然后 sec three sec four 有 join 那个 robocup\n'

In [263]:
text[19959] # gibberlish

'NI22FBP_0101\t668052\t669342\t他  对  她  有  ·䞠\x84\x9f枠§\x89\n'

In [264]:
text[20115] # loss of label: "#angpow" should be "#angpow#"

'NI22FBP_0101\t1040195\t1044030\tthen 他  会  给 ten dollar #angpow ha ha ha\n'

In [265]:
text[46226] # space between labeled term: if we use the default split method, #kim gary# will be split into 2 words
            # as the number of space between is 1 between english words and 2 between chinese words, it's really troublesome 
            # to do right split to avoid this problem

'UI17FAZ_0103\t751720\t754727\tah sorry我不吃#kim gary#的 呵呵呵\n'

In [266]:
text[21298] # % indicates a special noun

'NI24MBP_0101\t1138380\t1140790\t我  我  不是  去 %instep% 我是  去 g. i. p.\n'

In [267]:
text[21373] # % indicates a special noun

'NI25MBQ_0101\t583590\t587390\t所以 %zhi=hong% %zhi=hong% 有点 %zhi=hong% was very stress because er\n'

In [268]:
text[21503] # % indicates a special noun

'NI26FBP_0101\t1073610\t1076900\tit was then %sweety% decided that 他  会 take up cultural\n'

In [269]:
text[21513] # % indicates a special noun

'NI26FBP_0101\t1261440\t1262820\t回来  继续  玩 %viwawa%\n'

In [270]:
text[27698] # inconsistent annotation: according to the codebook, "sashimi" shoud be annotated as "#sashimi#" 
            # because it's a japanese word

'NI43FBP_0101\t1494840\t1501690\t不错  蛮  很好  吃  的  那边  的 %sashimi% 是  很  新鲜  的  那一  种  啊  就  一直  吃  "sashimi"\n'

In [271]:
text[48495] # wrongly split a word in transcription: "i. t." should be "IT", "o. k." should be "ok" in our sense 
            # it may help if we discard the "." after the character

'UI21MAZ_0101\t689491\t700786\t可能出来我发现我是不喜欢 i. t. 的 那时候如果现在是我工作我又有钱赚然后有可以得到经验如果又知道是 o. k. 的话到时半工读啰会比较好啦[mmm] \n'

In [272]:
text[54294] # need to check the audio to see if "epiko" is pronunced as a word or letters

'UI29FAZ_0102\t64704\t71545\t又有m.m.lah 我的我的有的有的我都不懂啊知道最著名的是e.p.i.k.o. [ah] h.r. [ah] err\n'

In [273]:
text[53626] # wrong transcription of mandarin: "大嫂干净" should be "打扫干净"

'UI28FAZ_0101\t221989\t230002\t大嫂干净[mmm] move 去那个新#taman#之后新的garden\n'

In [274]:
text[53390] # wrong transcription of english: "netbook" should be "notebook"

'UI27FAZ_0103\t1112733\t1119724\t啊touch screen的 and 通常会有人会买i. pad compared to netbook\n'

In [275]:
text[45496] # wrong annotation: "?" might refer to a non-transcribable sound?

'UI16MAZ_0103\t747433\t752878\t哦那个我还不能啦 但是我？了起来大约大约想想这样才可以啦 \n'

In [276]:
text[52996] # for this kind of unparsed utterance, we can split chinese charater and english words

'UI27FAZ_0101\t752877\t756798\tw. h. o.是world world health organize organization\n'

In [277]:
words_waste = []
for word in words:
    if word[-1] == "]" or word[-1] == ")" or word[-1] == "#":
        words_waste += [word]
        #print(word)

#print(words_waste)

d_waste = dict()
for word in words_waste:
    try:
        d_waste[word] += 1
    except:
        d_waste[word] = 1

len(words_waste)

12450

In [278]:
words_uniq = list(set(words))

In [279]:
len(words_uniq)

28614

In [280]:
words = [word.upper() for word in words]
d_train = dict()
for word in words:
    try:
        d_train[word] = d[word]
        for i in range(1,max_num+1):
            try:
                d_train[word+"(i)"] = d[word+"(i)"]
            except:
                break
    except:
        continue
print("Finish filtering")

Finish filtering


In [281]:
len(d_train)

8639